In [1]:
import torch.nn as nn

# CNN 모델 정의
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.fc1 = nn.Linear(128 * 32 * 32, 512)
        self.fc2 = nn.Linear(512, 2)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = self.pool(torch.relu(self.conv3(x)))
        x = x.view(-1, 128 * 32 * 32)
        x = torch.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

In [2]:
import torch
from torchvision import transforms
from PIL import Image, ImageTk
import os
import tkinter as tk
from tkinter import messagebox

# 모델 로드 함수
def load_model(model_path, device):
    model = CNNModel().to(device)  # CNNModel은 미리 정의된 모델 클래스입니다.
    model.load_state_dict(torch.load(model_path))
    model.eval()
    return model

# 예측 함수
def predict_image(model, image_path, device):
    transform = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    
    image = Image.open(image_path).convert('RGB')
    image = transform(image).unsqueeze(0).to(device)
    
    with torch.no_grad():
        output = model(image)
        _, predicted = torch.max(output, 1)
    return predicted.item()  # 0: real, 1: fake

# 게임 진행을 위한 GUI 클래스
class GameApp:
    def __init__(self, model, game_images_dir, device, num_rounds=5):
        self.model = model
        self.game_images_dir = game_images_dir
        self.device = device
        self.num_rounds = num_rounds
        self.round_num = 0
        self.score_user = 0
        self.score_model = 0
        self.image_paths = []
        self.labels = []  # 정답 라벨 (0: real, 1: fake)
        self.load_images()
        
        # tkinter GUI 설정
        self.root = tk.Tk()
        self.root.title("합성 이미지 맞추기 게임")
        
        # 이미지 표시용 라벨
        self.image_label = tk.Label(self.root)
        self.image_label.pack()
        
        
        # 'Real' 버튼
        self.real_button = tk.Button(self.root, text="합성X", command=lambda: self.check_answer("합성X"))
        self.real_button.pack(side=tk.LEFT, padx=20)
        
        # 'Fake' 버튼
        self.fake_button = tk.Button(self.root, text="합성0", command=lambda: self.check_answer("f합성0"))
        self.fake_button.pack(side=tk.LEFT, padx=20)
        
        # 게임 상태 표시
        self.status_label = tk.Label(self.root, text=f"Round {self.round_num + 1} / {self.num_rounds}")
        self.status_label.pack()
        
        # 게임 시작
        self.show_image()
        self.root.mainloop()

    # 이미지를 다운로드합니다.
    def load_images(self):
        for label in ['real', 'fake']:
            folder_path = os.path.join(self.game_images_dir, label)
            for img_name in os.listdir(folder_path):
                if img_name.endswith('.jpeg') or img_name.endswith('.jpg'):
                    self.image_paths.append(os.path.join(folder_path, img_name))
                    self.labels.append(0 if label == 'real' else 1)  # 합성 안한이미지는 0, 아니면 1
    
    def show_image(self):
        image_path = self.image_paths[self.round_num % len(self.image_paths)]
        self.image_path = image_path  # 현재 이미지 경로 저장
        
        # 이미지를 열고 보여줍니다.
        img = Image.open(image_path)
        img = img.resize((256, 256))  # Resize to fit the window
        img = ImageTk.PhotoImage(img)
        self.image_label.config(image=img)
        self.image_label.image = img  # Keep a reference to avoid garbage collection
        
        self.status_label.config(text=f"Round {self.round_num + 1} / {self.num_rounds}")

    # 사용자의 선택에 대하여 점수를 업데이트합니다.
    def check_answer(self, user_guess):
        true_label = self.labels[self.round_num % len(self.labels)]  # 실제 정답 (0: real, 1: fake)
        
        # 모델 예측
        model_prediction = predict_image(self.model, self.image_path, self.device)
        model_guess = '합성0' if model_prediction == 1 else "합성X"
        
        # 사용자 점수 업데이트
        if (user_guess == '합성X' and true_label == 0) or (user_guess == '합성0' and true_label == 1):
            self.score_user += 1
        
        # 모델 점수 업데이트
        if (model_guess == '합성X' and true_label == 0) or (model_guess == '합성0' and true_label == 1):
            self.score_model += 1
        
        # 결과 메시지
        result_message = f"Round {self.round_num + 1}\n"
        result_message += f"결과: 이미지 {'real' if true_label == 0 else 'fake'}\n"
        result_message += f"내 예측: {user_guess}, 모델 예측: {model_guess}\n"
        result_message += f"사용자 점수: {self.score_user}, 모델 점수: {self.score_model}"
        
        messagebox.showinfo("결과", result_message)
        
        # 다음 라운드로 넘어가기
        self.round_num += 1
        if self.round_num < self.num_rounds:
            self.show_image()  # 새로운 이미지 표시
        else:
            messagebox.showinfo("게임 종료", f"최종 점수:\n사용자: {self.score_user} | 모델: {self.score_model}")
            self.root.quit()

# 모델 불러오기
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_path = 'models/cnn_model_final.pth'  # 모델 경로
model = load_model(model_path, device)

# 게임 시작
game_images_dir = 'DataSet3/test'  # 게임에 사용할 이미지가 저장된 디렉토리
app = GameApp(model, game_images_dir, device, num_rounds=5)


C:\Users\Yeonsu\AppData\Local\Temp\ipykernel_25304\291651032.py:11: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path))
